In [17]:
import numpy as np                   # import numpy package under shorthand "np"
import pandas as pd                  # import pandas package under shorthand "pd"
import matplotlib.pyplot as plt
from nose.tools import assert_equal
from sklearn import preprocessing
from numpy.testing import assert_array_equal
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [4]:
df = pd.read_csv('noon_perfumes_dataset.csv')
df.head(5)
#droppin gunsued columns
data = df.drop(['brand', 'name','old_price','new_price','ml','concentration','item_rating','seller','seller_rating','num_seller_ratings'],axis=1)

date_c=[]
for i in data['base_note']:
    k=i.replace('and ', ', ')
    k=k.replace('And ', ', ')
    k=k.replace(' Notes ', '')
    k=k.lower()
    date_c.append(k)

date_s=[]
for i in data['scents']:
    k=i.replace('Oriental, Floral', 'Floral and Oriental')
    k=i.replace('Rose, Floral', 'Floral')
    date_s.append(k)
data.insert(0,"base",date_c)
data.insert(0,"scent",date_s)


print(data['scent'].unique())

1003
['Woody' 'Floral' 'Arabian' 'Spicy' 'Oriental' 'Fruity' 'Fresh' 'Citrus'
 'Vanilla' 'Musk' 'Oriental, Floral' 'Sweet Aromatic' 'Aromatic'
 'Woody, Sweet' 'Aromatic,Citrus' 'Clean' 'Woody, Musky'
 'Floral and Oriental' 'Jasmine' 'Woody And Spicy' 'Sandalwood']


In [3]:
data.head()

,scent,base,Unnamed: 0,department,scents,base_note,middle_note
0,Woody,"oakmoss, patchouli , vetiver",0,Men,Woody,"Oakmoss, Patchouli and Vetiver","Hazelnut, Jasmine, Cashmir Wood, Cedar and Honey"
1,Floral,"vanilla, sandalwood , patchouli",1,Women,Floral,"Vanilla, Sandalwood And Patchouli",Wild Jasmine and Red Lily
2,Arabian,"lemon, mint , wood moss",2,Unisex,Arabian,"Lemon, Mint and Wood Moss",Sandalwood and Cedar
3,Spicy,"cashmere wood, moss , rippled s, accord",3,Men,Spicy,"Cashmere Wood, Moss And Rippled Sand Accord",Blue Coral Aquaspace Accord And Geranium
4,Arabian,"vanille, benzoin, tonka bean",4,Women,Arabian,"Vanille, Benzoin, Tonka Bean",African Orange Flower


In [20]:
gh=df.groupby('name')
df['department'].value_counts().head(10)



department
Women          503
Men            433
Unisex          66
Kids Unisex      1
Name: count, dtype: int64

In [5]:
data2=data
data2.head()

,scent,base,Unnamed: 0,department,scents,base_note,middle_note
0,Woody,"oakmoss, patchouli , vetiver",0,Men,Woody,"Oakmoss, Patchouli and Vetiver","Hazelnut, Jasmine, Cashmir Wood, Cedar and Honey"
1,Floral,"vanilla, sandalwood , patchouli",1,Women,Floral,"Vanilla, Sandalwood And Patchouli",Wild Jasmine and Red Lily
2,Arabian,"lemon, mint , wood moss",2,Unisex,Arabian,"Lemon, Mint and Wood Moss",Sandalwood and Cedar
3,Spicy,"cashmere wood, moss , rippled s, accord",3,Men,Spicy,"Cashmere Wood, Moss And Rippled Sand Accord",Blue Coral Aquaspace Accord And Geranium
4,Arabian,"vanille, benzoin, tonka bean",4,Women,Arabian,"Vanille, Benzoin, Tonka Bean",African Orange Flower


In [6]:
data2['base']=data2['base'].str.replace(',',' ')
data2['base']=data2['base'].str.replace(', ',' ')
data2['base']=data2['base'].str.replace(' ,',' ')
data2['base']=data2['base'].str.replace('-',' ')
data2['base']=data2['base'].str.replace('.','')
data2['base']=data2['base'].str.replace(' notes',' note')
data2['base']=data2['base'].str.replace(' accords',' accord')
data2['base']=data2['base'].str.replace('dsy','dy')
data2['base']=data2['base'].str.replace('a ','')
data2['base']=data2['base'].str.replace('of ','')
data2['base']=data2['base'].str.replace('(oud)','oud')
data2['base']=data2['base'].str.replace(' and ','')
data2['base']=data2['base'].str.replace(' with ','')
#One hot encoding
asd=data2['base'].str.get_dummies(sep=' ')


,aandal,absolute,acaciwood,accord,addiction,agarwood,akigalawood,aldehyde,almod,almond,...,warm,whisky,white,whitemusk,wild,wisteria,woman,wood,woods,woody
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#getting labels
gender=pd.get_dummies(df['department'])

label_encoder = preprocessing.LabelEncoder()

gender2= label_encoder.fit_transform(df['department'])

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
#testing if the data types work for the ML model by just using the base notes as features
clf.fit(asd, gender2)
gender_pred=clf.predict(asd)
accuracy_score(gender2,gender_pred)

cv_results = cross_validate(clf,asd, gender2, cv=3)

/opt/software/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
